#Initialize Cell

In [ ]:
!pip install inflect
import inflect
p = inflect.engine()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Initialize Stanford NLP
!pip install stanza unidecode

import unidecode
import stanza
from itertools import permutations
from tqdm import tqdm
import pandas as pd
import re
from stanza.server import CoreNLPClient
import os

corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)
os.environ["CORENLP_HOME"] = corenlp_dir
!ls $CORENLP_HOME

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=13504ec08886475eeef120a8da73e848c3778f7971dcdcc92a236d003f868796
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


INFO:stanza:Installing CoreNLP package into ./corenlp


build.xml				  LIBRARY-LICENSES
corenlp.sh				  LICENSE.txt
CoreNLP-to-HTML.xsl			  Makefile
ejml-core-0.39.jar			  patterns
ejml-core-0.39-sources.jar		  pom-java-11.xml
ejml-ddense-0.39.jar			  pom-java-17.xml
ejml-ddense-0.39-sources.jar		  pom.xml
ejml-simple-0.39.jar			  protobuf-java-3.19.2.jar
ejml-simple-0.39-sources.jar		  README.txt
input.txt				  RESOURCE-LICENSES
input.txt.out				  sample-project-pom.xml
input.txt.xml				  SemgrexDemo.java
istack-commons-runtime-3.0.7.jar	  ShiftReduceDemo.java
istack-commons-runtime-3.0.7-sources.jar  slf4j-api.jar
javax.activation-api-1.2.0.jar		  slf4j-simple.jar
javax.activation-api-1.2.0-sources.jar	  stanford-corenlp-4.5.1.jar
javax.json-api-1.0-sources.jar		  stanford-corenlp-4.5.1-javadoc.jar
javax.json.jar				  stanford-corenlp-4.5.1-models.jar
jaxb-api-2.4.0-b180830.0359.jar		  stanford-corenlp-4.5.1-sources.jar
jaxb-api-2.4.0-b180830.0359-sources.jar   StanfordCoreNlpDemo.java
jaxb-impl-2.4.0-b180830.0438.jar	  StanfordDependenci

In [ ]:
client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner', 'depparse'], 
    memory='4G', 
    endpoint='http://localhost:9001',
    be_quiet=True)
client.start()

INFO:stanza:Writing properties to tmp file: corenlp_server-bc5d857426654a81.props
INFO:stanza:Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-bc5d857426654a81.props -annotators tokenize,ssplit,pos,lemma,ner,depparse -preload -outputFormat serialized


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import Data

In [ ]:
para_data_path = '/content/drive/MyDrive/Assets/Processed Dataset/Final_eval_10000/input_paragraph_train_2.csv'
ques_data_path = '/content/drive/MyDrive/Assets/Processed Dataset/Final_eval_10000/input_questions_train_10k.csv'
theme_path = '/content/drive/MyDrive/Assets/Processed Dataset/Final_eval_10000/theme_interval_train_10k.csv'
ground_truth_path = '/content/drive/MyDrive/Assets/Processed Dataset/Final_eval_10000/ground_truth_train_10k.csv'
para_df = pd.read_csv(para_data_path)
ques_df = pd.read_csv(ques_data_path)
theme_df = pd.read_csv(theme_path)
ground_truth_df = pd.read_csv(ground_truth_path)

In [ ]:
def paraLemma(df):
  data = dict()
  paragraphs = df.paragraph.tolist()
  lemma = []
  pos = []
  ner = []
  for para in paragraphs:
    lemma_str = ''
    pos_str = ''
    ner_str = ''
    document = client.annotate(para)
    for i, sent in enumerate(document.sentence):
      for t in sent.token:
        pos_str+=t.pos+' '
        ner_str+=t.ner+' '
        lemma_str+=t.lemma+' '
    lemma.append(lemma_str)
    pos.append(pos_str)
    ner.append(ner_str)
  data['paragraph'] = paragraphs
  data['lemma'] = lemma
  data['pos'] = pos
  data['ner'] = ner
  new_df = pd.DataFrame(data)
  new_df.to_csv('core_nlp_para.csv')
#paraLemma(para_df)

In [ ]:
lemma_path = '/content/drive/MyDrive/Assets/Processed Dataset/Final_eval_10000/core_nlp_para.csv'
lemma_df = pd.read_csv(lemma_path)

#Functions

In [ ]:
def apostrophe(question, paragraph):
  context = paragraph
  string = re.findall('"([^"]*)"', question) + re.findall("'([^']*)'", question)
  return string

def abbr(question):
  string = re.findall(r'(?:[A-Z]\.)+',question) + re.findall(r'\b(?:[A-Z]){2,}', question)
  return string

In [ ]:
def rule_based(question, paragraph, lemma_df):
  # Check for Abbreviation
  abbreviations = abbr(question)
  if len(abbreviations) > 0:
    for abbreviation in abbreviations:
      question = question.replace(abbreviation,'')
  question = question.lower()
  document = client.annotate(question)
  context = paragraph
  pos = []
  ner = []
  words = []
  normalized_ner = []
  coarse_ner = []
  lemma = []
  check_words = ['P1Y', 'PXY', 'PRESENT', 'PREV', 'NEXT', 'FUTURE', 'OFFSET', 'IMMEDIATE', 'THIS', 'PT1S', 'INTERSECT', 'P1D', 'P1M', '1985', 'P1W', 'TMO', 'TDT', 'PAST', 'TNI']
  flag = 0
  for i, sent in enumerate(document.sentence):
    for t in sent.token:
      words.append(t.word)
      pos.append(t.pos)
      ner.append(t.ner)
      normalized_ner.append(t.normalizedNER)
      lemma.append(t.lemma)
      coarse_ner.append(t.coarseNER)
  # Quotes
  if question.count('"')>1 or question.count("'")>1:
    narrow = []
    answer = apostrophe(question, context)
    for para in context:
      para_low=para.lower()
      if any(ans in para_low for ans in answer):
        narrow.append(para)
    if len(narrow)>0:
      context = list(set(narrow))
      flag = 1
    else:
      context = paragraph
  # NER
  if flag == 0:
    narrow = []
    if 'DATE' in ner:
      ner_indices = [idx for idx, value in enumerate(ner) if value == 'DATE']
      for ner_index in ner_indices:
        if not any(check_word in normalized_ner[ner_index] for check_word in check_words):
          word = words[ner_index]
          for para in context:
            para_low=para.lower()
            if word in para_low:
              narrow.append(para)
    if 'TIME' in ner:
      ner_indices = [idx for idx, value in enumerate(ner) if value == 'TIME']
      for ner_index in ner_indices:
        if not any(check_word in normalized_ner[ner_index] for check_word in check_words):
          word = words[ner_index]
          for para in context:
            para_low=para.lower()
            if word in para_low:
              narrow.append(para)
    if 'LOCATION' in ner: #ner -> coarse_ner
      ner_indices = [idx for idx, value in enumerate(ner) if value == 'LOCATION'] #ner -> coarse_ner
      for ner_index in ner_indices:
        word = words[ner_index]
        for para in context:
          para_low=para.lower()
          if word in para_low:
            narrow.append(para)
    if 'MONEY' in ner:
      ner_indices = [idx for idx, value in enumerate(ner) if value == 'MONEY']
      for ner_index in ner_indices:
        word = words[ner_index]
        for para in context:
          para_low=para.lower()
          if word in para_low:
            narrow.append(para)
    if 'ORGANIZATION' in ner:
      ner_indices = [idx for idx, value in enumerate(ner) if value == 'ORGANIZATION']
      for ner_index in ner_indices:
        word = words[ner_index]
        for para in context:
          para_low=para.lower()
          if word in para_low:
            narrow.append(para)
    if len(narrow)>0:
      context = list(set(narrow))
      flag = 1
    else:
      context = paragraph
  # PROPER NOUN
  if flag == 0 or len(context) == len(paragraph):
    for ind in range(-2, -len(pos), -1):
      if pos[ind] in ['NNP', 'NNPS']:
        word = words[ind]
        for para in context:
          para_low=para.lower()
          if word in para_low:
            narrow.append(para)
      else:
        break
    if len(narrow)>0:
      context = list(set(narrow))
      flag = 1
    else:
      context = paragraph
  # COMMON NOUN
  if flag == 0 or len(context)==len(paragraph):
    narrow = []
    if 'NN' in pos:
      pos_indices = [idx for idx, value in enumerate(pos) if value == 'NN']
      for pos_index in pos_indices:
        word = words[pos_index]
        if p.singular_noun(word):
          plural_word = p.plural_noun(word)
        else:
          plural_word = word
        for para in context:
          para_low=para.lower()
          if word in para_low or plural_word in para_low:
            narrow.append(para)
    if 'NNS' in pos:
      pos_indices = [idx for idx, value in enumerate(pos) if value == 'NNS']
      for pos_index in pos_indices:
        word = words[pos_index]
        if p.singular_noun(word):
          singular_word = p.singular_noun(word)
        else:
          singular_word = word
        for para in context:
          para_low=para.lower()
          if word in para_low or singular_word in para_low:
            narrow.append(para)
    if len(narrow)>0:
      context = list(set(narrow))
      flag = 1
    else:
      context = paragraph
  #VERB
  lemmatized_para = lemma_df[lemma_df.paragraph.isin(context)].lemma.tolist()
  if flag==0 or len(context)==len(paragraph):
    narrow = []
    if 'VB' in pos or 'VBP' in pos or 'VBZ' in pos or 'VBN' in pos or 'VBG' in pos or 'VBD' in pos:
      pos_indices = [idx for idx, value in enumerate(pos) if value == 'VB' or value == 'VBP' or value == 'VBZ' or value == 'VBN' or value == 'VBG' or value == 'VBD']
      for pos_index in pos_indices:
        word = lemma[pos_index]
        for para_id,para in enumerate(lemmatized_para):
          para_low=para.lower()
          if word in para_low:
            narrow.append(context[para_id])
    if len(narrow)>0:
      context = list(set(narrow))
      flag = 1
    else:
      context = paragraph
  return context

#Main Test Loop

In [ ]:
wrong_pred_list = []
accuracy_data = []
acc_total = 0
blank_total = 0
no_narrow_total = 0
wrong_total = 0
length_of_narrow = []
length_of_narrow_perc = []
for theme in tqdm(theme_df.theme):
  accuracy_dict = dict()
  accuracy_dict['theme'] = theme
  questions = ques_df[ques_df.theme==theme].question.tolist()
  acc = 0
  blank_pred = 0
  no_narrow = 0
  wrong_pred = 0
  for question in questions:
    real_para = ground_truth_df[ground_truth_df.question_id == ques_df[ques_df.question == question].iloc[0,0]].iloc[0, -1] + 1
    real_ans = ground_truth_df[ground_truth_df.question_id == ques_df[ques_df.question == question].iloc[0,0]].iloc[0, -2]
    paras = para_df[para_df.theme==theme].paragraph.tolist()
    max_thres = len(paras)/3
    min_thres = len(paras)/6
    trim_question = unidecode.unidecode(question)
    lemmatized_para = lemma_df[lemma_df.paragraph.isin(paras)]
    ans = rule_based(trim_question, paras, lemmatized_para)
    length_of_narrow.append(len(ans))
    length_of_narrow_perc.append(len(ans)/len(paras))
    pred_para = para_df[para_df.paragraph.isin(ans)].id.tolist()
    if len(pred_para) == 0 and not real_ans=='[]':
      blank_pred+=1
      blank_total+=1
    elif len(pred_para) == len(paras) and not real_ans=='[]':
      no_narrow+=1
      no_narrow_total+=1
    elif real_para in pred_para or real_ans=='[]':
      acc+=1
      acc_total+=1
    else:
      wrong_pred_list.append(question)
      wrong_pred+=1
      wrong_total+=1
  acc = acc/len(questions)
  blank_pred = blank_pred/len(questions)
  accuracy_dict['acc'] = acc
  accuracy_dict['blank_pred'] = blank_pred
  accuracy_data.append(accuracy_dict)
  acc_total, no_narrow_total, blank_total, wrong_total

100%|██████████| 46/46 [09:29<00:00, 12.38s/it]


In [ ]:
acc_total, no_narrow_total, blank_total, wrong_total

(9437, 131, 0, 432)

In [ ]:
ques_df['length_of_narrow'] = length_of_narrow
ques_df['length_of_narrow_perc'] = length_of_narrow_perc

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Assets/train_data.csv')
train_data[train_data.Question.isin(wrong_pred_list)].to_csv('./wrong_pred_424.csv', index=False, header=True)

In [ ]:
test = client.annotate('What movie did Beyonce act in 2006?')
for sen in test.sentence:
  for tok in sen.token:
    print(tok)

word: "What"
pos: "WDT"
value: "What"
before: ""
after: " "
originalText: "What"
ner: "O"
lemma: "what"
beginChar: 0
endChar: 4
tokenBeginIndex: 0
tokenEndIndex: 1
hasXmlContext: false
isNewline: false
coarseNER: "O"
fineGrainedNER: "O"
nerLabelProbs: "O=0.9999708045725951"

word: "movie"
pos: "NN"
value: "movie"
before: " "
after: " "
originalText: "movie"
ner: "O"
lemma: "movie"
beginChar: 5
endChar: 10
tokenBeginIndex: 1
tokenEndIndex: 2
hasXmlContext: false
isNewline: false
coarseNER: "O"
fineGrainedNER: "O"
nerLabelProbs: "O=0.9997124582603977"

word: "did"
pos: "VBD"
value: "did"
before: " "
after: " "
originalText: "did"
ner: "O"
lemma: "do"
beginChar: 11
endChar: 14
tokenBeginIndex: 2
tokenEndIndex: 3
hasXmlContext: false
isNewline: false
coarseNER: "O"
fineGrainedNER: "O"
nerLabelProbs: "O=0.9999300453298651"

word: "Beyonce"
pos: "NNP"
value: "Beyonce"
before: " "
after: " "
originalText: "Beyonce"
ner: "PERSON"
lemma: "Beyonce"
beginChar: 15
endChar: 22
tokenBeginIndex: 3
to

In [ ]:
client.stop()